<a href="https://colab.research.google.com/github/GadirajuSanjayvarma/EVA4/blob/master/EVA4-QUIZ/EVA4_S9_QUIZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import datetime

x = datetime.datetime.now()
print(x)

2020-03-17 13:45:28.646165


In [1]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-en06reap
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-en06reap
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64514 sha256=cc8d9af7958b363af5073379bcbab7b27210f7244006b1a5c44d7d9a6781cc69
  Stored in directory: /tmp/pip-ephem-wheel-cache-spzukvt1/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.4.5
    Uninstalling albumentations-0.4.5:
      Successfully uninstalled albumentations-0.4.5


In [2]:
# mount the drive where we will save stats and loade our libraries from
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# add path to our library in default search path
import sys
sys.path.append('/content/drive/My Drive/EVA4')

In [0]:
class album_compose:
  def __init__(self):
    self.albumentations_transform=A.Compose([
                           A.Rotate(limit=(-15,+15),p=0.3),
                           A.HorizontalFlip(p=0.3),
                           A.Normalize(mean=list(channel_means),std=list(channel_stdevs)), 
                            A.Cutout(num_holes=4,fill_value=list(channel_means)),               
                           A.pytorch.ToTensor()

    ])


  def __call__(self,img):
    img=np.array(img)
    img=self.albumentations_transform(image=img)['image']
    return img  

In [0]:
import numpy as np
import albumentations as A
import eva4datatransforms
from torchvision import transforms
channel_means = (0.49139968, 0.48215841, 0.44653091)
channel_stdevs = (0.24703223, 0.24348513, 0.26158784)
trans = eva4datatransforms.Transforms(normalize=True, mean=channel_means, stdev=channel_stdevs)
fillmeans = (np.array(channel_means)*255).astype(np.uint8)
# Train Phase transformations

train_transforms =album_compose() 

# Test Phase transformations
test_transforms =trans.test_transforms()


In [6]:
from torchvision import datasets
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import eva4dataloaders

dataloader = eva4dataloaders.DataLoader()
# train dataloader
train_loader = dataloader.load(train)

# test dataloader
test_loader = dataloader.load(test)

In [8]:
import eva4models
import QuizDNN
import torch
model = QuizDNN.QuizNet()
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
model.to(model.device)
model.summary((3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]              96
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]           9,216
       BatchNorm2d-5           [-1, 32, 32, 32]              64
              ReLU-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 32, 32, 32]           9,216
       BatchNorm2d-8           [-1, 32, 32, 32]              64
              ReLU-9           [-1, 32, 32, 32]               0
        MaxPool2d-10           [-1, 32, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]           2,048
      BatchNorm2d-12           [-1, 64, 16, 16]             128
             ReLU-13           [-1, 64, 16, 16]               0
           Conv2d-14           [-1, 64,

In [9]:
#from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
EPOCHS = 8
L2lambda = 0
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=False, weight_decay=L2lambda)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)
L1lambda = 0
model.gotrain(optimizer, train_loader, test_loader, EPOCHS, "/content/drive/My Drive",scheduler, True, L1lambda)

Validation loss decreased (100000.000000 --> 1.359004).  Saving model ...
Epoch: 1, Train set: Average loss: 0.0119, Accuracy: 43.19%; Test set: Average loss: 1.3590, Accuracy: 51.33%
Learning Rate = 0.198207


Validation loss decreased (1.359004 --> 1.301939).  Saving model ...
Epoch: 2, Train set: Average loss: 0.0087, Accuracy: 59.81%; Test set: Average loss: 1.3019, Accuracy: 56.37%
Learning Rate = 0.468180


Validation loss decreased (1.301939 --> 0.841056).  Saving model ...
Epoch: 3, Train set: Average loss: 0.0071, Accuracy: 68.07%; Test set: Average loss: 0.8411, Accuracy: 70.52%
Learning Rate = 0.485852


Validation loss decreased (0.841056 --> 0.760829).  Saving model ...
Epoch: 4, Train set: Average loss: 0.0061, Accuracy: 72.57%; Test set: Average loss: 0.7608, Accuracy: 73.80%
Learning Rate = 0.405592


Validation loss decreased (0.760829 --> 0.692760).  Saving model ...
Epoch: 5, Train set: Average loss: 0.0053, Accuracy: 76.36%; Test set: Average loss: 0.6928, Accuracy: 76.22%
Learning Rate = 0.277636


Validation loss decreased (0.692760 --> 0.544314).  Saving model ...
Epoch: 6, Train set: Average loss: 0.0047, Accuracy: 78.94%; Test set: Average loss: 0.5443, Accuracy: 81.69%
Learning Rate = 0.141207


Validation loss decreased (0.544314 --> 0.483442).  Saving model ...
Epoch: 7, Train set: Average loss: 0.0040, Accuracy: 82.09%; Test set: Average loss: 0.4834, Accuracy: 83.78%
Learning Rate = 0.038130


Validation loss decreased (0.483442 --> 0.457229).  Saving model ...
Epoch: 8, Train set: Average loss: 0.0035, Accuracy: 84.55%; Test set: Average loss: 0.4572, Accuracy: 84.52%
Learning Rate = 0.000002

